In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ta
import requests
import pandas as pd

# Set maximum number of columns to display
pd.set_option('display.max_columns', None)  # None means no limit

# Set maximum width for each column
pd.set_option('display.max_colwidth', None)  # None means no limit

# Set maximum width for the entire DataFrame
pd.set_option('display.width', None)  # None means no limit

## Stock Price History

* Pulls Stock data from yahoo finance api
* Add technical analysis features to the stock history

### API Calls

In [2]:
# # Prepare credential parameter for api fetching
# symbol = 'NVDA'
# api_key = '0WZYWLZ2BC8HSUZ5'
# url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={api_key}'

# # Request fuel data given the  prepared credential
# response = requests.request("GET", url)

# # ==================== #
# # Succesful api status #
# # ==================== #
# if response.status_code == 200:

#     # Store data as json file
#     data = response.json()
        
# else:
#     print('API request failed')

In [3]:
# # Prepare credential parameter for api fetching
# symbol = 'NVDA'
# interval =  'daily'
# period = 8
# api_key = '0WZYWLZ2BC8HSUZ5'
# url = f'https://www.alphavantage.co/query?function=EMA&symbol={symbol}&interval={interval}&time_period={period}&series_type=close&apikey={api_key}'

# # Request fuel data given the  prepared credential
# response = requests.request("GET", url)

# # ==================== #
# # Succesful api status #
# # ==================== #
# if response.status_code == 200:

#     # Store data as json file
#     ema_8_data = response.json()
        
# else:
#     print('API request failed')

### Yahoo Finance Library Call

In [4]:
from pymongo import MongoClient

# Connect to the MongoDB server (default host and port)
client = MongoClient('mongodb://localhost:27017/')

# Create or access a database named 'stock_data_db'
db = client['stock_data_db']

# Create or access a collection named 'stock_prices'
collection = db['stock_prices_history']

In [5]:
# Fundemental Good Stocks
stock_symbols = [
    "NVDA", "TSLA", "META", "GOOGL", "PLTR", "GOOG", "BRK.B", "BRK.A", 
    "FSLR", "BSX", "GOLD", "EA", "INTU", "SHOP", "ADI", "RMD", 
    "ISRG", "ANET", "VRTX", "WELL", "WAB", "O", "AEM", "VICI", 
    "XYL", "IR", "AME", "VEEV", "FTV", "INFY", "KHC", "SAP", 
    "GRMN", "CP", "TCOM", "ALC", "TW", "EQR", "FAST", "ERIE",
    "TRI", "GIB", "CHT"  
]

In [6]:
# Historic Stock Price
def get_price(symbol):
    x = yf.Ticker(f"{symbol}")
    x = x.history(period='max').reset_index()
    x.loc[:, 'Date'] = x['Date'].dt.strftime('%Y-%m-%d')
    x = x.loc[:, ['Date', 'Open', 'High', 'Low', 'Close','Volume']]
    return x

# Get the Symbol
price_hist = get_price('TSLA')
price_hist

# Add Technical Indicators
def add_technical(values):
    
    # Add ema dual channels technical indicators
    values['8EMA'] = ta.trend.ema_indicator(values['Close'], window=8)
    values['13EMA'] = ta.trend.ema_indicator(values['Close'], window=13)
    values['144EMA'] = ta.trend.ema_indicator(values['Close'], window=144)
    values['169EMA'] = ta.trend.ema_indicator(values['Close'], window=169)

    # Add MACD technical indicator
    values['MACD'] = ta.trend.macd(values['Close'], window_slow=26, window_fast=12)
    values['MACD_SIGNAL'] = ta.trend.macd_signal(values['Close'], window_slow=26, window_fast=12, window_sign=9)
    values['MACD_HIST'] = ta.trend.macd_diff(values['Close'], window_slow=26, window_fast=12, window_sign=9)
    
    # Add ATR technical indicator
    values["atr"] = ta.volatility.AverageTrueRange(high=values.High, low=values.Low, close=values.Close).average_true_range()
    values["atr"] = values.atr.rolling(window=30).mean()
    return values

price_hist = add_technical(price_hist)

,Date,Open,High,Low,Close,Volume
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500
...,...,...,...,...,...,...
3575,2024-09-12 00:00:00-04:00,224.660004,231.449997,223.830002,229.809998,72020000
3576,2024-09-13 00:00:00-04:00,228.000000,232.669998,226.320007,230.289993,59515100
3577,2024-09-16 00:00:00-04:00,229.300003,229.960007,223.529999,226.779999,54323000
3578,2024-09-17 00:00:00-04:00,229.449997,234.570007,226.550003,227.869995,66761600


## Data Cleaning

In [9]:
def clean(df):
    # Reset Index as date
    df = df.set_index('Date')
    # Drop the rows with missing values
    for row in range(len(df)):
        if any(pd.isnull(df.iloc[row,0:5])):
            df = df.drop(row)
        else:
            continue
        
    # Filter the data
    df = df.iloc[-500:,:]
    return df

price_hist = clean(price_hist)

## Features Engineering

In [10]:
class features_engineer:
    def __init__(self, df):
        self.df = df.copy()

    def add_candlestick(self):
        self.df["BodyDiff"] = abs(self.df["Open"] - self.df["Close"])
        self.df["CandleStickType"] = np.where(self.df["Open"] < self.df["Close"], "Green", "Red")
        return self.df

    def continuous_increase(self, windows=3):
        for i in range(1, windows + 1):
            self.df[f"Close_t-{i}"] = self.df["Close"].shift(i)

        self.df['Incremental_High'] = (self.df['Close'] > self.df['Close_t-1']) \
                                        & (self.df['Close_t-1'] > self.df['Close_t-2']) \
                                        & (self.df['Close_t-2'] > self.df['Close_t-3'])
        return self.df

    def macd_golden_cross(self):
        self.df['MACD_GOLDEN_CROSS'] = (self.df['MACD'] > self.df['MACD_SIGNAL']) & (self.df['MACD'] < 0)
        return self.df

    def add_ema_band(self, threshold=0.05):
        self.df['169EMA_Upper'] = self.df['169EMA'] * (1 + threshold)
        self.df['169EMA_Lower'] = self.df['169EMA'] * (1 - threshold)
        return self.df

    def process_all(self):
        self.add_candlestick()
        self.continuous_increase()
        self.macd_golden_cross()
        self.add_ema_band()
        return self.df

price_hist = features_engineer(price_hist).process_all()

KeyError: 'MACD'

In [11]:
price_hist

,Open,High,Low,Close,Volume
Date,,,,,
2022-09-22 00:00:00-04:00,299.859985,301.290009,285.820007,288.589996,70545400
2022-09-23 00:00:00-04:00,283.089996,284.500000,272.820007,275.329987,63748400
2022-09-26 00:00:00-04:00,271.829987,284.089996,270.309998,276.010010,58076900
2022-09-27 00:00:00-04:00,283.839996,288.670013,277.510010,282.940002,61925200
2022-09-28 00:00:00-04:00,283.079987,289.000000,277.570007,287.809998,54664800
...,...,...,...,...,...
2024-09-12 00:00:00-04:00,224.660004,231.449997,223.830002,229.809998,72020000
2024-09-13 00:00:00-04:00,228.000000,232.669998,226.320007,230.289993,59515100
2024-09-16 00:00:00-04:00,229.300003,229.960007,223.529999,226.779999,54323000


## Technical Strategy

### Tecnical Signal

In [ ]:
class Strategy:
    
    def __init__(self, df):
        self.df = df.copy()
        self.df['MACD_Alert'] = -1  
        self.df['Engulf_Alert'] = -1 
        self.df['dual_channel_Alert'] = -1  
        self.df['382_Alert'] = -1  
        self.window = 3  # Number of days to compare the stock price
        
    def engulf_alert(self):
        
        # Previous Candle
        prev_open = self.df['Open'].shift(1)
        prev_close = self.df['Close'].shift(1)
        
        # Current Candle
        current_open = self.df['Open']
        current_close = self.df['Close']

        # Bullish Engulfing: current green candle engulfs previous red candle
        bullish_engulfing = (
            (prev_close < prev_open) &  # Previous candle was red
            (current_close > current_open) &  # Current candle is green
            (current_open < prev_close) & 
            (current_close > prev_open)
        )

        # Bearish Engulfing: current red candle engulfs previous green candle
        bearish_engulfing = (
            (prev_close > prev_open) &  # Previous candle was green
            (current_close < current_open) &  # Current candle is red
            (current_open > prev_close) & 
            (current_close < prev_open)
        )

        # Apply the conditions to the DataFrame
        self.df.loc[bullish_engulfing, 'Engulf_Alert'] = 1
        self.df.loc[bearish_engulfing, 'Engulf_Alert'] = 0
        
        return self.df

    def macd_alert(self):
        
        # Pre-compute conditions
        macd_above_signal = self.df['MACD'] > self.df['MACD_SIGNAL']
        macd_increasing = self.df['MACD'].diff() > 0
        macd_below_zero = self.df['MACD'] >= 0

        # Bullish condition
        bullish_macd = (
            macd_above_signal &
            macd_increasing&
            macd_below_zero
        )
        
        # Bearish condition
        bearish_macd = (
            (self.df['MACD'] < self.df['MACD_SIGNAL']) &
            (self.df['MACD'] > self.df['MACD'].shift(-1))  
        )

        # Apply the conditions to the DataFrame
        self.df.loc[bullish_macd, 'MACD_Alert'] = 1
        self.df.loc[bearish_macd, 'MACD_Alert'] = 0

        return self.df

    def dual_channel(self):
        
        # Confirm Technical Indicators Conditions
        ema_8_gt_ema_13 = self.df['8EMA'] > self.df['13EMA']
        ema_13_gt_ema_169 = self.df['13EMA'] > self.df['169EMA']
        ema_8_gt_ema_144 = self.df['8EMA'] > self.df['144EMA']
        
        # Confirm Close price action
        close_ge_ema_13 = self.df['Close'] >= self.df['13EMA']
        close_ge_ema_144 = self.df['Close'] >= self.df['144EMA']
        
        # Confirm Candle fully above 13 EMA
        low_ge_ema_13 = self.df['Low'] >= self.df['13EMA'] 
        green_candle = self.df['CandleStickType'] == 'Green'    
        
        # Confirm Candle in selected range
        open_in_slow_ema = self.df['Open'].between(self.df['169EMA_Lower'], self.df['169EMA_Upper'])
        
        # Confirm Candle type
        green_candle = self.df['CandleStickType'] == 'Green'
        
        # Bullish Conditions
        
        bullish_scenario_1 = (
            ema_8_gt_ema_13 & 
            ema_13_gt_ema_169 & 
            close_ge_ema_13 &
            low_ge_ema_13 &
            green_candle
        )
        
        # Bearish Condition
        bearish_scenario_1 = (
            (self.df['Open'] < self.df['13EMA']) &
            (self.df['Close'] < self.df['13EMA']) &
            (self.df['13EMA'] < self.df['8EMA'])
        )
        # Bullish Conditions 2
        bullish_scenario_2 = ( 
            ema_8_gt_ema_144 &
            ema_13_gt_ema_169 &
            close_ge_ema_144 &
            open_in_slow_ema &
            green_candle
            )
        
        # Bearish Condition 2
        bearish_scenario_2 = (
            (self.df['Open'] < self.df['13EMA']) &
            (self.df['Close'] < self.df['13EMA']) &
            (self.df['8EMA'] < self.df['13EMA']) &
            ~ open_in_slow_ema
        )

        # Apply conditions on different price scenarios
        
        self.df.loc[bullish_scenario_1 | bullish_scenario_2, 'dual_channel_Alert'] = 1
        self.df.loc[bearish_scenario_1 | bearish_scenario_2, 'dual_channel_Alert'] = 0
        
        return self.df

    def bullish_382_alert(self):
        # Calculate Fibonacci 38.2% retracement level
        diff = self.df['High'] - self.df['Low']
        fib_382_level = self.df['High'] - 0.382 * diff
        open_above_fib_382 = self.df['Open'] > fib_382_level
        
        # Apply condition
        self.df.loc[open_above_fib_382, '382_Alert'] = 1
        self.df.loc[~open_above_fib_382, '382_Alert'] = 0
        
        return self.df

    def add_alert(self):
        self.engulf_alert()
        self.macd_alert()
        self.dual_channel()
        self.bullish_382_alert()
        return self.df
    
# Apply the strategy
strategy_df = Strategy(price_hist).add_alert()

print(f"Bullish MACD Count: {len(strategy_df[strategy_df['MACD_Alert'] == 1])} | Bearish Count: {len(strategy_df[strategy_df['MACD_Alert'] == 0])}")
print(f"Bullish Engulf Count: {len(strategy_df[strategy_df['Engulf_Alert'] == 1])} | Bearish Engulf Count: {len(strategy_df[strategy_df['Engulf_Alert'] == 0])}")
print(f"Bullish 382 Count: {len(strategy_df[strategy_df['382_Alert'] == 1])} | Bearish 382 Count: {len(strategy_df[strategy_df['382_Alert'] == 0])}")
print(f"Bullish Dual Channel Count: {len(strategy_df[strategy_df['dual_channel_Alert'] == 1])} | Bearish Dual Channel Count: {len(strategy_df[strategy_df['dual_channel_Alert'] == 0])}")


## Trade Back Testing

### Based Case Trading

In [ ]:
total_values = 10000
value_investment_results = (1 + (strategy_df['Close'].iloc[-1] - strategy_df['Open'].iloc[0]) / strategy_df['Close'].iloc[0]) * total_values
print(f"Value Investment Total Return: {value_investment_results}")

### Strategic Trading

In [ ]:
trades = []
current_trade = {}

for date in range(len(strategy_df)-1):
    
    try:
        # Sell Trade
        if len(current_trade) != 0:
            if (strategy_df['dual_channel_Alert'].iloc[date] == 0)\
                or date == len(strategy_df) - 1:
                    # Has Trade and Bearish
                    
                trades.append(
                    {   
                    "Entry_price":current_trade["entry_price"],
                    "Entry_date":current_trade["entry_date"],
                    "Exit_price":strategy_df['Open'].iloc[date],
                    "Exit_date":strategy_df.index[date],
                    "profit":(strategy_df['Open'].iloc[date]/current_trade['entry_price']) - 1,
                    "total_asset":(total_values*(strategy_df.iloc[date].Open/current_trade['entry_price']) - 1)
                    }
                )
                # Update total asset
                total_values = (total_values*(strategy_df.iloc[date+1].Open/current_trade['entry_price']))
                
                # Close Trade
                current_trade = {}

        # Buy Trade
        elif (strategy_df['dual_channel_Alert'].iloc[date] == 1) \
            and (len(current_trade) == 0): # No Trade and Bullish

            current_trade["entry_price"] = strategy_df['Close'].iloc[date]
            current_trade["entry_date"] = strategy_df.index[date]
            
    except IndexError:
        print("No Trade Data") 
    
total_values = 10000

trades = pd.DataFrame(trades)

strategic_results = trades.iloc[-1].total_asset
print(f"Technical Strategic Total Return: {strategic_results}")
strategic_results = 0

In [ ]:
trades

## Dash Interactive

In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.subplots as sp

# Assuming you have your data in a pandas DataFrame named 'price_hist'

# Initialize Dash app
app = dash.Dash(__name__)

# Create figure with subplots
fig = sp.make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.1, row_heights=[0.65, 0.15,0.25])

# Candlestick chart
fig.add_trace(go.Candlestick(
    x=price_hist.index,
    open=price_hist['Open'],
    high=price_hist['High'],
    low=price_hist['Low'],
    close=price_hist['Close'],
    name='Price'), row=1, col=1)

# Add EMA traces as lines
fig.add_trace(go.Scatter(x=price_hist.index, y=price_hist['144EMA'], 
                         mode="lines", name="EMA 144"),row=1,col=1)
fig.add_trace(go.Scatter(x=price_hist.index, y=price_hist['169EMA'],
                         mode="lines", name="EMA 169"),row=1,col=1)
fig.add_trace(go.Scatter(x=price_hist.index, y=price_hist['13EMA'],
                         mode="lines", name="EMA 13"),row=1,col=1)
fig.add_trace(go.Scatter(x=price_hist.index, y=price_hist['8EMA'],
                         mode="lines", name="EMA 8"),row=1,col=1)

# Add Buy / Sell Annoation
fig.add_trace(go.Scatter(
                x=trades.Entry_date,
                y=trades.Entry_price,
                mode = "markers",
                customdata=trades,
                marker_symbol="diamond-dot",
                marker_size = 8,
                marker_line_width = 2,
                marker_line_color = "rgba(0,0,0,0.7)",
                marker_color="rgba(0,255,0,0.7)",
                hovertemplate="Entry Time: %{customdata[1]}<br>" +\
                    "Entry Price: %{y:.2f}<br>" +\
                    "Total Asset: %{customdata[5]:.3f}",
                name="Entries"),row=1, col=1)

fig.add_trace(go.Scatter(
                x=trades.Exit_date,
                y=trades.Exit_price,
                mode = "markers",
                customdata=trades,
                marker_symbol="diamond-dot",
                marker_size = 8,
                marker_line_width = 2,
                marker_line_color = "rgba(0,0,0,0.7)",
                marker_color="rgba(255,0,0,0.7)",
                hovertemplate="Exit Time: %{customdata[1]}<br>" +\
                    "Exit Price: %{y:.2f}<br>" +\
                    "Total Asset: %{customdata[5]:.3f}",
                name="Exits"),row=1, col=1)

# Add MACD subplot
fig.add_trace(go.Scatter(x=price_hist.index, y=price_hist['MACD'], 
                         mode='lines', name='MACD'), row=2, col=1)
fig.add_trace(go.Scatter(x=price_hist.index, y=price_hist['MACD_SIGNAL'],
                         mode='lines', name='MACD signal'), row=2, col=1)

# Add Profit subplot
fig.add_trace(go.Scatter(x=trades['Exit_date'], y=trades['total_asset'], 
                         mode='lines', name='Profit'), row=3, col=1)
# Layout settings
fig.update_layout(
    xaxis_rangeslider_visible=False,
    autosize=False,
    width=1000,  # Width of the figure in pixels
    height=800,  # Height of the figure in pixels
)
# Define the layout of the Dash app
app.layout = html.Div([
    dcc.Graph(id='macd-graph', figure=fig)
])

# Callback to update y-axis and x-axis range based on selected range
@app.callback(
    Output('macd-graph', 'figure'),
    Input('macd-graph', 'relayoutData')  # Listen for relayout events (zoom, pan, etc.)
)
def update_axes_range(relayout_data):
    # If a new x-axis range is selected or zoomed
    if relayout_data and 'xaxis.range[0]' in relayout_data and 'xaxis.range[1]' in relayout_data:
        x_start = relayout_data['xaxis.range[0]']
        x_end = relayout_data['xaxis.range[1]']

        # Filter the data based on the selected x-axis range
        filtered_data = price_hist[(price_hist.index >= x_start) & (price_hist.index <= x_end)]
        trades_filtered = trades[(trades['Exit_date'] >= x_start) & (trades['Exit_date'] <= x_end)]
        
        # Calculate the new y-axis range for the candlestick chart
        y_min = filtered_data['Low'].min()
        y_max = filtered_data['High'].max()

        # Calculate the new y-axis range for the MACD chart
        y_min_macd = filtered_data[['MACD', 'MACD_SIGNAL']].min().min()
        y_max_macd = filtered_data[['MACD', 'MACD_SIGNAL']].max().max()

        # Calculate the new y-axis range for the Profit chart
        y_min_profit = trades_filtered['total_asset'].min()
        y_max_profit = trades_filtered['total_asset'].max()

        # Update the figure with new y-axis and x-axis ranges
        fig.update_xaxes(range=[x_start, x_end], row=1, col=1)
        fig.update_xaxes(range=[x_start, x_end], row=2, col=1)  
        fig.update_xaxes(range=[x_start, x_end], row=3, col=1) 
        
        fig.update_yaxes(range=[y_min, y_max], row=1, col=1)
        fig.update_yaxes(range=[y_min_macd, y_max_macd], row=2, col=1)
        fig.update_yaxes(range=[y_min_profit, y_max_profit], row=3, col=1)

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
